# Introducción

¡Bienvenido al curso de aprendizaje automático intermedio de Kaggle!

Si tienes experiencia en aprendizaje automático y te gustaría aprender cómo mejorar rápidamente la calidad de tus modelos, ¡estás en el lugar correcto! En este curso, acelerará su experiencia en aprendizaje automático aprendiendo a:

. Abordar los tipos de datos que a menudo se encuentran en conjuntos de datos del mundo real (valores faltantes, variables categóricas),

. Diseñar canales para mejorar la calidad de su código de aprendizaje automático,

. Utilizar técnicas avanzadas para la validación de modelos (validación cruzada),

. Construir modelos de última generación que se utilizan ampliamente para ganar competencias de Kaggle (XGBoost), y

. Evite errores comunes e importantes en la ciencia de datos (fugas).

A lo largo del camino, aplicará sus conocimientos completando un ejercicio práctico con datos del mundo real para cada tema nuevo. Los ejercicios prácticos utilizan datos de la Competencia de precios de vivienda para usuarios de Kaggle Learn, donde utilizará 79 variables explicativas diferentes (como el tipo de techo, la cantidad de dormitorios y la cantidad de baños) para predecir los precios de las viviendas. ¡Medirás tu progreso enviando predicciones a esta competencia y observando cómo asciende tu posición en la clasificación!


### Requisitos previos

Está listo para este curso si ya ha creado un modelo de aprendizaje automático y está familiarizado con temas como la validación de modelos, el ajuste insuficiente y excesivo, y los bosques aleatorios.

Si es completamente nuevo en el aprendizaje automático, consulte nuestro curso de Introducción al aprendizaje automático, que cubre todo lo que necesita para prepararse para este curso.

# Ejercicios

Como calentamiento, revisará algunos fundamentos del aprendizaje automático y enviará sus resultados iniciales a una competencia de Kaggle.

### Configuración

Las preguntas a continuación le brindarán comentarios sobre su trabajo. Ejecute la siguiente celda para configurar el sistema de retroalimentación.

In [ ]:
# Set up code checking
import os
if not os.path.exists("../input/train.csv"):
    os.symlink("../input/home-data-for-ml-course/train.csv", "../input/train.csv")  
    os.symlink("../input/home-data-for-ml-course/test.csv", "../input/test.csv")  
from learntools.core import binder
binder.bind(globals())
from learntools.ml_intermediate.ex1 import *
print("Setup Complete")

Trabajará con datos de la Competencia de precios de vivienda para usuarios de Kaggle Learn para predecir los precios de las viviendas en Iowa utilizando 79 variables explicativas que describen (casi) todos los aspectos de las viviendas.

Ejecute la siguiente celda de código sin cambios para cargar las funciones de entrenamiento y validación en X_train y X_valid, junto con los objetivos de predicción en y_train e y_valid. Las funciones de prueba se cargan en X_test. (Si necesita revisar características y objetivos de predicción, consulte este breve tutorial. Para leer sobre la validación de modelos, mire aquí. Alternativamente, si prefiere ver un curso completo para revisar todos estos temas, comience aquí. )

In [ ]:
# Read the data
X_full = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# Obtain target and predictors
y = X_full.SalePrice
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = X_full[features].copy()
X_test = X_test_full[features].copy()

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

Utilice la siguiente celda para imprimir las primeras filas de los datos. Es una buena manera de obtener una descripción general de los datos que utilizará en su modelo de predicción de precios.

In [ ]:
X_train.head()

La siguiente celda de código define cinco modelos de bosque aleatorios diferentes. Ejecute esta celda de código sin cambios. (Para revisar bosques aleatorios, mire aquí).

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Define the models
model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='absolute_error', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]

Para seleccionar el mejor modelo de los cinco, definimos una función score_model() a continuación. Esta función devuelve el error absoluto medio (MAE) del conjunto de validación. Recuerde que el mejor modelo obtendrá el MAE más bajo. (Para revisar el error absoluto medio, consulte aquí).

Ejecute la celda de código sin cambios.

In [ ]:
from sklearn.metrics import mean_absolute_error

# Function for comparing different models
def score_model(model, X_t=X_train, X_v=X_valid, y_t=y_train, y_v=y_valid):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)

for i in range(0, len(models)):
    mae = score_model(models[i])
    print("Model %d MAE: %d" % (i+1, mae))

### Paso 1: evaluar varios modelos

Utilice los resultados anteriores para completar la línea a continuación. ¿Qué modelo es el mejor modelo? Su respuesta debe ser una de las siguientes: modelo_1, modelo_2, modelo_3, modelo_4 o modelo_5.

In [ ]:
# Fill in the best model
best_model = model_3

# Check your answer
step_1.check()

### Paso 2: generar predicciones de prueba

Excelente. Sabe cómo evaluar qué hace que un modelo sea preciso. Ahora es el momento de pasar por el proceso de modelado y hacer predicciones. En la línea siguiente, cree un modelo de bosque aleatorio con el nombre de variable my_model.

In [ ]:
# Define a model
my_model = best_model # Your code here

# Check your answer
step_2.check()

Ejecute la siguiente celda de código sin cambios. El código ajusta el modelo a los datos de entrenamiento y validación y luego genera predicciones de prueba que se guardan en un archivo CSV. ¡Estas predicciones de prueba se pueden enviar directamente a la competencia!

In [ ]:
# Fit the model to the training data
my_model.fit(X, y)

# Generate test predictions
preds_test = my_model.predict(X_test)

# Save predictions in format used for competition scoring
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)